# SGCN species view

This notebook provides the necessary queries to execute the [species view](https://www1.usgs.gov/csas/swap/species_view.html) page in the reworked SWAP application. It runs a query based on a given species display name. The scientificname_display property in the SGCN data is based on either the final accepted name from the taxonomic authority or the cleaned up original scientific name for cases where the authority did not return a value. The display name serves as a functional key in the data to return information pertinent to taxa in the system.

In [42]:
import requests
from ipywidgets import widgets
from IPython.display import display

gc2BaseSQLAPIURL = "https://gc2.mapcentia.com/api/v1/sql/bcb"

This function takes a scientificname_display value and searches via the sgcn table in the GC2 instance SQL API for states by year reported. The results can be used through an appropriate mapping engine to display a US map with states and territories highlighted appropriately.

As with other pages for the new reworked SWAP app, it would be really great if the new system can respond to URL parameters so that we can simply link to these species pages with the scientific name in the URL.

In [43]:
def searchName(sender):
    speciesSearchSQL = "SELECT state, sgcnyear, taxonomicauthorityid_accepted \
        FROM sgcn \
        WHERE scientificname_display = '"+speciesName.value+"' \
        ORDER BY sgcnyear,state"
    gc2SearchURL = gc2BaseSQLAPIURL+"?q="+speciesSearchSQL
    gc2SearchResults = requests.get(gc2SearchURL).json()
    
    # Show the JSON output from the states query
    print ("SGCN state results by year for "+speciesName.value)
    display(gc2SearchResults)

    # Get the taxonomic authority ID (if applicable) and run the ITIS check function
    x = 0
    taxonomicauthorityid_accepted = ""
    for feature in gc2SearchResults['features']:
        taxonomicauthorityid_accepted = feature['properties']['taxonomicauthorityid_accepted']
        x = x + 1
        if x > 0:
            break

    # Fire off the function to get information for the species from the TIR Cache
    getTIRCache(taxonomicauthorityid_accepted,speciesName.value)
    
    # Fire the function to get states not yet reporting
    sgcnStatesNotReporting()

# States not reporting
We still need to report on the list of states that have not yet submitted reports. This can be achieved from the data by querying for states or territories that had reported in 2005 but are not yet in the 2015 data.

In [47]:
def sgcnStatesNotReporting():
    statesNotReportingSQL = "SELECT DISTINCT state \
        FROM sgcn \
        WHERE sgcnyear = 2005 \
        AND state NOT IN \
        (SELECT DISTINCT state FROM sgcn WHERE sgcnyear=2015)"
    gc2StatesNotReportingURL = gc2BaseSQLAPIURL+"?q="+statesNotReportingSQL
    gc2StatesNotReporting = requests.get(gc2StatesNotReportingURL).json()
    print ("Query result for states not yet reporting")
    display(gc2StatesNotReporting)

# Taxonomic Information Registry
I've gone ahead and built out a data source called tircache in the GC2 instance and developed a couple of scripts to retrieve selected information from the FWS ECOS TESS system and the NatureServe web services into that store. Tircache is basically a simple triple store of identifiers tied to values through a rudimentary predicate system based on URN syntax. This isn't the most elegant way to handle the information, but it can serve the basic purpose while we work out a better solution.

The following sections deal with three types of information:
* Common names for species
* Federal listing status from FWS based on ITIS TSN
* NatureServe status information for US and states based on scientific name lookup and global ID response

Not all of the species names and IDs have been run, so this data will not yet be complete. However, it shows the basic response information that can be incorporated with some conditional logic on the species view page in the SWAP app.

The query looks for all information from the tircache table for a given species based on either taxonomic authority ID or scientific name (both of which are used as keys/subject IDs).

Note: I need to rework this just a bit once I start grabbing information from WoRMS, but it doesn't impact anything functional at this point.

In [48]:
def getTIRCache(itisId,speciesName):
    sqlITISCommonNameSearch = "SELECT subjectid, property, value FROM tircache \
        WHERE subjectid = '"+itisId+"' OR subjectid = '"+speciesName+"'"
    urlITISCommonNameSearch = gc2BaseSQLAPIURL+"?q="+sqlITISCommonNameSearch
    itisCommonNames = requests.get(urlITISCommonNameSearch).json()
    print ("Information about the species from the Taxonomic Information Registry")
    display (itisCommonNames)

# Species Search
Use this widget to fire off the functions needed to pull information for a given species.

In [49]:
speciesName = widgets.Text(description="Species Name")
display(speciesName)

speciesName.on_submit(searchName)

SGCN state results by year for Accipiter gentilis


{'_execution_time': 0.111,
 'auth_check': {'auth_level': 'Write',
  'checked_relations': ['sgcn'],
  'session': None,
  'success': True},
 'features': [{'properties': {'sgcnyear': 2005,
    'state': 'Alaska',
    'taxonomicauthorityid_accepted': 'http://services.itis.gov/?q=tsn:175300'},
   'type': 'Feature'},
  {'properties': {'sgcnyear': 2005,
    'state': 'California',
    'taxonomicauthorityid_accepted': 'http://services.itis.gov/?q=tsn:175300'},
   'type': 'Feature'},
  {'properties': {'sgcnyear': 2005,
    'state': 'Colorado',
    'taxonomicauthorityid_accepted': 'http://services.itis.gov/?q=tsn:175300'},
   'type': 'Feature'},
  {'properties': {'sgcnyear': 2005,
    'state': 'Connecticut',
    'taxonomicauthorityid_accepted': 'http://services.itis.gov/?q=tsn:175300'},
   'type': 'Feature'},
  {'properties': {'sgcnyear': 2005,
    'state': 'Maryland',
    'taxonomicauthorityid_accepted': 'http://services.itis.gov/?q=tsn:175300'},
   'type': 'Feature'},
  {'properties': {'sgcnyear

Information about the species from the Taxonomic Information Registry


{'_execution_time': 0.101,
 'auth_check': {'auth_level': 'Write',
  'checked_relations': ['tircache'],
  'session': None,
  'success': True},
 'features': [],
 'forGrid': [{'dataIndex': 'subjectid',
   'header': 'subjectid',
   'type': 'character varying',
   'typeObj': None},
  {'dataIndex': 'property',
   'header': 'property',
   'type': 'character varying',
   'typeObj': None},
  {'dataIndex': 'value',
   'header': 'value',
   'type': 'character varying',
   'typeObj': None}],
 'forStore': [{'name': 'subjectid', 'type': 'character varying'},
  {'name': 'property', 'type': 'character varying'},
  {'name': 'value', 'type': 'character varying'}],
 'success': True,
 'type': 'FeatureCollection'}

Query result for states not yet reporting


{'_execution_time': 0.119,
 'auth_check': {'auth_level': 'Write',
  'checked_relations': ['sgcn'],
  'session': None,
  'success': True},
 'features': [{'properties': {'state': 'Alabama'}, 'type': 'Feature'},
  {'properties': {'state': 'Tennessee'}, 'type': 'Feature'},
  {'properties': {'state': 'Georgia'}, 'type': 'Feature'},
  {'properties': {'state': 'Louisiana'}, 'type': 'Feature'},
  {'properties': {'state': 'California'}, 'type': 'Feature'},
  {'properties': {'state': 'New Mexico'}, 'type': 'Feature'},
  {'properties': {'state': 'New Hampshire'}, 'type': 'Feature'},
  {'properties': {'state': 'American Samoa'}, 'type': 'Feature'},
  {'properties': {'state': 'Hawaii'}, 'type': 'Feature'},
  {'properties': {'state': 'Connecticut'}, 'type': 'Feature'},
  {'properties': {'state': 'Alaska'}, 'type': 'Feature'},
  {'properties': {'state': 'Oklahoma'}, 'type': 'Feature'},
  {'properties': {'state': 'Idaho'}, 'type': 'Feature'},
  {'properties': {'state': 'West Virginia'}, 'type': 'Featu